In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Disease = candidate_subclass('Disease', ['disease'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load candidates

### NOTE: Need to take into account the 6% we miss!!!

In [2]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates').one()
print len(train)
dev = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates').one()
print len(dev)

28087
27896


In [3]:
d = train[0]
d

OperationalError: (OperationalError) no such column: disease.disease_cid u'SELECT disease.disease_id AS disease_disease_id, disease.disease_cid AS disease_disease_cid \nFROM disease \nWHERE ? = disease.id' (1,)

#### Load gold labels

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

L_gold_train = label_manager.load(session, train, "CDR Training Label Set")
print L_gold_train.shape
L_gold_dev = label_manager.load(session, dev, "CDR Development Label Set")
print L_gold_dev.shape

# Testing LFs

## Idea 3: How to filter single words?

In [ ]:
train[0]

In [ ]:
neg_singles = []
pos_singles = []
for i,c in enumerate(dev):
    if len()

## Debugging dictionaries

In [ ]:
from cPickle import load
from utils import load_mesh_raw

MESH_to_CID = load(open('MESH_to_CID.pkl', 'rb'))
mesh_entries = load_mesh_raw('data/desc2017.xml')

In [ ]:
mesh_tree = defaultdict(list)
for entry in mesh_entries:
    mid, tree_nums, terms = entry
    for tn in tree_nums:
        path = [tn[0]] + tn[1:].split(".")
        for term in terms:
            mesh_tree[term].append((mid, path))

In [ ]:
mid_to_paths = defaultdict(list)
for entry in mesh_entries:
    mid, tree_nums, terms = entry
    for tn in tree_nums:
        path = [tn[0]] + tn[1:].split(".")
        mid_to_paths[mid].append(path)

In [ ]:
len(mesh_tree)

In [ ]:
with open('cui2mesh.tsv', 'rb') as f:
    for line in f:
        term, cui, mid = line.rstrip('\n').split('\t')
        for path in mid_to_paths[mid]:
            x = (mid, path)
            t = term.lower()
            if x not in mesh_tree[t]:
                mesh_tree[t].append(x)

In [ ]:
len(mesh_tree)

In [ ]:
mesh_tree[0]

### Idea 2: Try exact partial matches

To start, anchored on the last word

In [ ]:
from cPickle import load
from utils import load_mesh_raw

MESH_to_CID = load(open('MESH_to_CID.pkl', 'rb'))
mesh_entries = load_mesh_raw('data/desc2017.xml')

mesh_tree = defaultdict(list)
for entry in mesh_entries:
    mid, tree_nums, terms = entry
    for tn in tree_nums:
        path = [tn[0]] + tn[1:].split(".")
        for term in terms:
            mesh_tree[term].append((mid, path))

In [ ]:
POS_DEPTH = 2
NEG_DEPTH = 1

def LFG_MESH_exact_partial_end_anchored(c):
    toks = c.disease.get_attrib_tokens()
    for i in range(1, len(toks)):
        p = " ".join(toks[i:])
        if p in mesh_tree :
            seen = set()
            for mid, path in mesh_tree[p]:
                value = MESH_to_CID[mid] if path[0] in ['C', 'F'] else -1
                key   = "_".join(path[:POS_DEPTH]) if value > 0 else "_".join(path[:NEG_DEPTH])
                if key not in seen:
                    seen.add(key)
                    yield key, value
            break

In [ ]:
for c in train

In [ ]:
%time L_train = label_manager.create(session, train, 'LF Training Labels -- MESH exact end match test', f=LFG_MESH_exact_partial_end_anchored)
L_train

In [ ]:
L_train.lf_stats(labels=L_gold_train)

### Idea 1: Vote no on JJ / ending in JJ

Note: if this is very good, just adjust the candidate extractor? **--> TRYING THIS NOW...**

In [ ]:
from snorkel.models import Label

c = train[0]
print c
print c.disease.get_attrib_tokens('pos_tags')
session.query(Label).filter(Label.candidate == c).one()

In [ ]:
def no_jj_only(c):
    pos_tags = c.disease.get_attrib_tokens('pos_tags')
    if len(pos_tags) == 1 and pos_tags[0] == 'JJ':
        return -1
    else:
        return 0

def no_jj_ending(c):
    pos_tags = c.disease.get_attrib_tokens('pos_tags')
    if pos_tags[-1] == 'JJ':
        return -1
    else:
        return 0

In [ ]:
LFs = [no_jj_only, no_jj_ending]
%time L_train = label_manager.create(session, train, 'LF Training Labels -- No JJ Test', f=LFs)
L_train

In [ ]:
L_train.lf_stats(labels=L_gold_train)

In [ ]:
fns = []
for i,c in enumerate(train):
    if L_gold_train[i] > 0 and no_jj_only(c) == -1:
        fns.append(c)
print len(fns)